In [1]:
# import necessary libraries
import mesa
import seaborn as sns
import pandas as pd
import shutil

# import cacheable model wrapper
from cacheable_model import CacheableModel

The following code is to clear the default directory which the cached fiels are saved to.

In [2]:
directory = "output_dir"

def delete_directory(directory_path):
    shutil.rmtree(directory_path)
    print(f"Directory {directory_path} and its contents have been deleted.")

try:
    delete_directory(directory)
except:
    pass

Directory output_dir and its contents have been deleted.


Define the Boltzmann Wealth Model. I chose to reimplement it here so that I can modify the attributes and experiment with the results more easily. The code is taken from the tutorial

In [3]:
def compute_gini(model):
    agent_wealths = [agent.wealth for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum(xi * (N - i) for i, xi in enumerate(x)) / (N * sum(x))
    return 1 + (1 / N) - 2 * B


class MoneyAgent(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        cellmates.pop(
            cellmates.index(self)
        )  # Ensure agent is not giving money to itself
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1
            if other == self:
                print("I JUST GAVE MONEY TO MYSELF HEHEHE!")

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()


class MoneyModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, N, width, height):
        super().__init__()
        self.num_agents = N
        self.grid = mesa.space.MultiGrid(width, height, True)
        self.schedule = mesa.time.RandomActivation(self)

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.datacollector = mesa.DataCollector(
            model_reporters={"Gini": compute_gini}, agent_reporters={"Wealth": "wealth"}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

Instantiate the base model without caching

In [4]:
model = MoneyModel(100, 10, 10)

In [5]:
def condition_function(model_vars): # condition to cache the results specifically
        return model_vars.get('Gini', 0)[-1] > 0.7
    
number_of_steps = 1000

cacheable_model = CacheableModel(model, directory, number_of_steps, 100, condition_function=condition_function)

In [6]:
for i in range(number_of_steps):
    cacheable_model.model.step()
    cacheable_model.get_grid_dataframe("test")
    cacheable_model.cache()

Condition met. Appended special results for step 80 to output_dir/special_results.parquet
Condition met. Appended special results for step 245 to output_dir/special_results.parquet
Condition met. Appended special results for step 246 to output_dir/special_results.parquet
Condition met. Appended special results for step 247 to output_dir/special_results.parquet
Condition met. Appended special results for step 248 to output_dir/special_results.parquet
Condition met. Appended special results for step 249 to output_dir/special_results.parquet
Condition met. Appended special results for step 254 to output_dir/special_results.parquet
Condition met. Appended special results for step 257 to output_dir/special_results.parquet
Condition met. Appended special results for step 258 to output_dir/special_results.parquet
Condition met. Appended special results for step 259 to output_dir/special_results.parquet
Condition met. Appended special results for step 260 to output_dir/special_results.parquet


In [7]:
model_df, agent_df = cacheable_model.combine_dataframes()
model_df, agent_df

(       Gini
 0    0.0000
 1    0.2112
 2    0.3842
 3    0.5126
 4    0.5110
 ..      ...
 995  0.6900
 996  0.6712
 997  0.6400
 998  0.6346
 999  0.5856
 
 [1000 rows x 1 columns],
               Wealth
 Step AgentID        
 0    0             1
      1             1
      2             1
      3             1
      4             1
 ...              ...
 999  95            0
      96            0
      97            1
      98            0
      99            3
 
 [100000 rows x 1 columns])

In [12]:
cacheable_model.read_grid_state("test", cacheable_model)

KeyError: 'width'

In [ ]:
import pandas as pd
from typing import List, Any

def read_grid_state(filename='grid_cache.parquet', model: Any = None) -> Any:
    """Read the grid state from a Parquet file and reconstruct the grid with agents."""
    # Load DataFrame from Parquet
    df = pd.read_parquet(filename)
    
    # Extract grid dimensions from the file
    width = int(df['width'].iloc[0])
    height = int(df['height'].iloc[0])
    
    # Initialize the grid
    grid = Grid(width=width, height=height)  # Adjust as needed for your grid class

    # Reconstruct agents and place them in the grid
    agents = df[['pos_x', 'pos_y', 'wealth', 'unique_id']]
    for _, row in agents.iterrows():
        pos_x = int(row['pos_x'])
        pos_y = int(row['pos_y'])
        unique_id = int(row['unique_id'])
        wealth = float(row['wealth'])
        
        # Create agent instance (adjust based on how you create agents)
        agent = Agent(unique_id=unique_id, model=model)
        agent.pos = (pos_x, pos_y)
        agent.wealth = wealth

        # Place agent in the grid
        grid._grid[pos_x][pos_y] = grid._grid[pos_x][pos_y] or []
        grid._grid[pos_x][pos_y].append(agent)

    return grid


In [ ]:
# Example of loading a grid from a Parquet file
  # Your model instance, or a placeholder if required
grid = read_grid_state('test', model=model)


In [ ]:
df = pd.read_parquet("test")
df